# Read SeaWIFs 8 day extracted data

this R notebook reads netcdf files from L3 Browser @ NASA Ocean Color https://oceancolor.gsfc.nasa.gov/l3/

the data is from 8-day running averages of PAR measured by SeaWIFs extracted from the Cariaco Region

In [1]:
library("ncdf4")
library("reshape2")

In [2]:
outputfile <- "DATA/SeaWiFS_monthly.csv"
if (file.exists(outputfile)) file.remove(outputfile)    # caution new data will be appended to this file if it already exists
print(outputfile)

[1] TRUE

[1] "DATA/SeaWiFS_monthly.csv"


## create bounding box & create list of file names

In [3]:
# list and remove objects
ls()
rm(list = ls())

# set the study area
latmax <- 10.683
latmin <- 10.492
lonmax <- -64.367
lonmin <- -64.735


#lat = c(10.35,  10.78, 10.78, 10.35)
#lon = c(-64.53,-64.53,-64.9, -64.9)

#latmax <- 10.8
#latmin <- 10.5
#lonmax <- -64.6
# lonmin <- -65.2

# create a list of files and indicate its length
f <- list.files("RAW/requested_files_4/.", pattern="*.L3m_MO_CHL.x_chlor_a.nc",full.names=F)
lf<-length(f)
print(head(f))

[1] "outputfile"

[1] "S19972741997304.L3m_MO_CHL.x_chlor_a.nc"
[2] "S19973051997334.L3m_MO_CHL.x_chlor_a.nc"
[3] "S19973351997365.L3m_MO_CHL.x_chlor_a.nc"
[4] "S19980011998031.L3m_MO_CHL.x_chlor_a.nc"
[5] "S19980321998059.L3m_MO_CHL.x_chlor_a.nc"
[6] "S19980601998090.L3m_MO_CHL.x_chlor_a.nc"


## show netcdf file structure

In [4]:
data<-nc_open(paste('RAW/requested_files_4/',f[1],sep=''))
print(data)
#ncvar_get(data, "chlor_a")
# extract data
nc_close(data)

File RAW/requested_files_4/S19972741997304.L3m_MO_CHL.x_chlor_a.nc (NC_FORMAT_NETCDF4):

     2 variables (excluding dimension variables):
        float chlor_a[lon,lat]   (Chunking: [3,2])  (Compression: shuffle,level 4)
            long_name: Chlorophyll Concentration, OCI Algorithm
            units: mg m^-3
            standard_name: mass_concentration_of_chlorophyll_in_sea_water
            _FillValue: -32767
            valid_min: 0.00100000004749745
            valid_max: 100
            reference: Hu, C., Lee Z., and Franz, B.A. (2012). Chlorophyll-a algorithms for oligotrophic oceans: A novel approach based on three-band reflectance difference, J. Geophys. Res., 117, C01011, doi:10.1029/2011JC007395.
            display_scale: log
            display_min: 0.00999999977648258
            display_max: 20
        unsigned byte palette[eightbitcolor,rgb]   (Contiguous storage)  

     4 dimensions:
        lat  Size:5
            long_name: Latitude
            units: degrees_nort

## define variable to extract 

In [5]:
# variable
#var <- "chlor_a"

## Open each individual file and extract values within bounding box defined above

In [6]:
for (i in 1:lf) {
  # progress indicator
  print(paste("Processing file",i,"from",length(f),sep=" "))
  # open netCDF file
  data<-nc_open(paste('RAW/requested_files_4/',f[i],sep=''))
  # extract data
  lon<-ncvar_get(data,"lon")
  lat<-ncvar_get(data,"lat")
  value<-ncvar_get(data,"chlor_a")
  unit<-ncatt_get(data,"chlor_a","units")$value
  # matrix to data.frame
  dimnames(value)<-list(lon=lon,lat=lat)
  dat.var<-melt(value,id="lon")
  # select data from the study area taking out missing data
  dat.varSAtmp<-dat.var #subset(dat.var,lon<=lonmax & lon>=lonmin & lat<=latmax & lat>=latmin)
  # extract date information
  dateini<-ncatt_get(data,0,"time_coverage_start")$value
  dateend<-ncatt_get(data,0,"time_coverage_end")$value
  #print(c("ini", dateini, "end", dateend))
  datemean<-mean(c(as.Date(dateend,"%Y-%m-%dT%H:%M:%OSZ"),as.Date(dateini,"%Y-%m-%dT%H:%M:%OSZ")))
  year<- substring(datemean,0,4) #1997 + ((i+6)/12)#
  month<- substring(datemean,6,7) #i+6 #
  day<- substring(datemean,9,10)
  # prepare final data set
  dat.varSA<-data.frame(rep(as.integer(year,nrow(dat.varSAtmp))),rep(as.integer(month,nrow(dat.varSAtmp))),rep(as.integer(day,nrow(dat.varSAtmp))),
                        dat.varSAtmp,rep(unit,nrow(dat.varSAtmp)),rep("chlor_a",nrow(dat.varSAtmp)))
  names(dat.varSA)<-c("year","month","day","lon","lat","value","unit","var")
  # save csv file DATA/FirstTestExtracted.csv
  fe<-file.exists(paste('DATA/','SeaWiFS_monthly.csv',sep=''))
  write.table(dat.varSA,paste('DATA/','SeaWiFS_monthly.csv',sep=''),row.names=FALSE,col.names=!fe,sep=",",dec=".",append=fe)
  # close connection
  nc_close(data)
  # clean workspace
  rm(data,lon,lat,value,unit,dat.var,dat.varSAtmp,dateini,dateend,datemean,year,month,dat.varSA,fe)
}
rm(f,i,latmax,latmin,lf,lonmax,lonmin)

[1] "Processing file 1 from 152"
[1] "Processing file 2 from 152"
[1] "Processing file 3 from 152"
[1] "Processing file 4 from 152"
[1] "Processing file 5 from 152"
[1] "Processing file 6 from 152"
[1] "Processing file 7 from 152"
[1] "Processing file 8 from 152"
[1] "Processing file 9 from 152"
[1] "Processing file 10 from 152"
[1] "Processing file 11 from 152"
[1] "Processing file 12 from 152"
[1] "Processing file 13 from 152"
[1] "Processing file 14 from 152"
[1] "Processing file 15 from 152"
[1] "Processing file 16 from 152"
[1] "Processing file 17 from 152"
[1] "Processing file 18 from 152"
[1] "Processing file 19 from 152"
[1] "Processing file 20 from 152"
[1] "Processing file 21 from 152"
[1] "Processing file 22 from 152"
[1] "Processing file 23 from 152"
[1] "Processing file 24 from 152"
[1] "Processing file 25 from 152"
[1] "Processing file 26 from 152"
[1] "Processing file 27 from 152"
[1] "Processing file 28 from 152"
[1] "Processing file 29 from 152"
[1] "Processing file 30